In [1]:
import pandas as pd 
import psycopg2

In [2]:
conn_params = {
    'host': 'etl-relational-database.cfmnnswnfhpn.eu-west-2.rds.amazonaws.com',
    'dbname': 'postgres',
    'user': 'postgres-readonly',
    'password': 'ZY>[#4Bmx0AFDOkE'
}

analytical_database_conn_params = {
    'host': 'localhost',
    'dbname': 'etl_bites',
    'user': '',
    'password': ''
}


In [3]:
def fetch_data(query):
    with psycopg2.connect(**conn_params) as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            data = cur.fetchall()
            colnames = [desc[0] for desc in cur.description]
    return data, colnames


def execute_query_postgresql(query):
    with psycopg2.connect(**conn_params) as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            conn.commit()



In [ ]:
# Extract

extract_sql_query = """SELECT facid, slots FROM bookings;"""

bookings_data, colnames = fetch_data(extract_sql_query)
bookings_df = pd.DataFrame(bookings_data, columns=colnames)

bookings_df['booking_duration'] = bookings_df['slots'] * 30
total_booking_duration_per_facility = bookings_df.groupby('facid')['booking_duration'].sum().reset_index()
total_booking_duration_per_facility.columns = ['FacilityID', 'TotalBookingDurationInMinutes']

total_booking_duration_per_facility

In [ ]:
create_total_duration_table = '''
CREATE TABLE IF NOT EXISTS total_booking_duration (
    FacilityID INTEGER NOT NULL,
    TotalBookingDurationInMinutes INTEGER NOT NULL
);
'''

execute_query_postgresql(create_total_duration_table)

In [ ]:
def insert_data(table_name, data, columns):
    with psycopg2.connect(**conn_params) as conn:
        with conn.cursor() as cur:
            for row in data.itertuples(index=False):
                insert_query = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({', '.join(['%s'] * len(columns))});"
                cur.execute(insert_query, row)
            conn.commit()

# Insert the transformed data into the analytical database
insert_data('booking_duration', total_booking_duration_per_facility, ['FacilityID', 'TotalBookingDurationInMinutes'])